# From Docker to Kubernetes with Helm

Docker has been a huge win for deploying software across the board and in particular for deploying machine learning models from my own experience. Perhaps you've already adopted this practice but wonder how to take your container and deploy it at scale. This post seeks to address that question exactly: How to take the next step from Dockerizing your application to deploying it with Kubernetes.

## Who this post is for

While my specific experience is deploying models in this way, this post is general enough to cover what you need to know for any developer to make the transition. That being said this post is geared towards *developers* not maintainers of a Kubernetes cluster. Here we'll cover the basics that a developer needs to know to get started such as

- How to create your fist helm chart that utilizes an existing Docker repository.
- How to test the chart on your local workstation.
- Gotchas you may encounter following the Helm tutorial and some troubleshooting tips.

In particular we'll show how to build a container around a simple [flask](flask.pocoo.org) app and write a Helm chart that enables us to run the app on Kubernetes.

Things we won't cover:

- The ins and outs of all that Helm is and what it can do
- How to stand up a real (not local) kubernetes cluster

# Why Kubernetes?

If you are reading this post you likely already know the answer to this question. However we'll give a brief answer here anyway. Kubernetes is an application originally developed within Google (that has since been open sourced) for deploying containers at scale. Kubernetes has huge support from both the software community and every major cloud provider and therefore going to kubernetes as a solution for deploying containers is an obvious choice.

# Why Helm?

Helm was introduced to standardize how Kubernetes applications are "packaged." The anology that the project uses is that Helm is to `yum` or `apt-get` as Helm charts (the data about your project that the `helm` executable consumes) are analogous to `rpm` or `deb` files.

For the developer who wants to quickly get started with Kubernetes Helm is a great solution. There are many moving pieces and configurations to a Kubernetes app and Helm provides a great way of encapsulating and version controlling all of these details. In this way it is a bit like `docker-compose`, i.e. a single place where you can define your configurations and coordinate services.

# Taking the step

Now we'll discuss how to take your Docker container and getting it running on Kubernetes with Helm.

To get started you'll need to download [minikube](https://github.com/kubernetes/minikube) (which lets you run a local kubernetes cluster) and [helm](https://github.com/kubernetes/helm). Note that when initializing `helm` you may need to specify the service account as default with `helm init --upgrade --service-account default`. If you simply use `helm init --upgrade` and run into issues later (such as bouncing back and forth between the errors "Error: no available release name found" and "Error: could not find a ready tiller pod") run this command

```shell
kubectl delete deployment tiller-deploy --namespace kube-system
```

and then initialize `helm` using service account default as described above.

We're now ready to follow the [official helm quickstart tutorial](https://docs.bitnami.com/kubernetes/how-to/create-your-first-helm-chart/). As we go along I'll point out a few gotchas.

Before getting started we'll need minikube up and running, fire it up with

```shell
minikube start
```

## Creating the Helm chart

`helm create` is the simplest way to create a Helm chart for beginners. It is also recommended as a best practice by the Helm developers as it guarantees you'll be starting your project off with the correct chart structure. Let's use the same chart name from the tutorial

```shell
helm create mychart
```

You should now have a `mychart` directory. This directory is the Helm chart for your project. Installing the chart is simple, on the command line run

```shell
helm install --name example ./mychart --set service.type=NodePort
```

Our project is now installed to our `minikube` cluster. To verify that is working, in your terminal run

```shell
minikube service example
```

The "Welcome to nginx!" page should have opened in your browser.

We didn't do much, but let's summarize what just happened.

1. We fired up a local kubernetes cluster.
2. We used the built in `helm create` tool to give us a basic Helm chart template. This default template simply runs an `nginx` server.
3. We installed the Helm chart to the cluster with `helm install`. The service type was set to `NodePort` to expose the service outside of the cluster. The tutorial contains for details on this.
4. We verified that we could interact with the service.

# Customizing the Helm chart

Now let's customize the chart around a simple flask app.

If you'd like to follow along with the commands here, clone my GitHub repository (link at bottom of page).

First we'll create a new chart as before

```shell
helm create from-docker-to-kubernetes-chart
```

Now we'll build the container (the container needs to exist on our local machine before isntalling the helm chart. You can pull images from Dockerhub or a private registry, but for this example we'll keep it simple)

```shell
docker build -t from-docker-to-kubernetes:stable .
```

Now container isn't anything special. It's simply a flask app that runs on port 5000 inside the container and returns a randomly generated number every time you visit the root (`/`) endpoint.

To get the chart functioning properly we need to make 3 changes only

1. Specify our custom docker image.
2. Specify which port inside the container kubernetes should listen to.

To make the first change we edit the lines in `from-docker-to-kubernetes-chart/values.yml` from

```yml
image:
  repository: nginx
  tag: stable
```

to

```yml
image:
  repository: from-docker-to-kubernetes
  tag: stable
```

To make the second two changes we add the lines to `values.yml`

```yml
container:
  # changed from 80
  port: 5000
```

Now we can install our app with

```shell
helm install --name from-docker-to-kubernetes ./from-docker-to-kubernetes-chart --set service.type=NodePort
```

## Gotcha: liveness and readiness probes

In this case we have properly installed our flask app. However we overlooked a key element that tripped me up when I was writing my first helm chart.

In `templates/deployments.yml` "liveness" and "readiness" probes are defined. These are descriptions of how to ping the service to know if it is "alive." By default the example in `helm create` simply makes an HTTP GET request to the '/'. If it returns 200 the checks are considered successful. If not, the service will continually be restarted until the check passes.

I was able to figure this out buy reading [this blog post](https://kukulinski.com/10-most-common-reasons-kubernetes-deployments-fail-part-1/) which has greate trouble shooting advice for working with kubernetes.